In [1]:
%pwd

'/content'

In [2]:
%cd drive/MyDrive/transformer/lightning_transformer/

/content/drive/MyDrive/transformer/lightning_transformer


In [3]:
%pip install datasets
%pip install tokenizers
%pip install torchmetrics
%pip install pytorch_lightning


In [ ]:
# For pytorch training
# from config import get_config

# cfg = get_config()

# cfg['batch_size'] = 16
# cfg['preload'] = None
# cfg['num_epochs'] = 10

# from train import train_model

# train_model(cfg)

Using device: cuda
Max length of source sentence: 309
Max length of target sentence: 274


Processing Epoch 08:  93%|█████████▎| 1698/1819 [22:38<01:36,  1.26it/s, loss=5.304]

In [3]:
import torch
import pytorch_lightning as pl

print(f"Torch={torch.__version__}, Lightening={pl.__version__}")

Torch=2.0.1+cu118, Lightening=2.0.8


In [4]:
from config import config
from datamodule import BilingualDataModule

data_module = BilingualDataModule(
    dataset_path=config.dataset_path,
    src_lang=config.lang_src,
    tgt_lang=config.lang_tgt,
    seq_len=config.seq_len,
    batch_size=config.batch_size,
    num_workers=config.num_workers,
    pin_memory=config.pin_memory,
    tokenizer_file=config.tokenizer_file,
)

data_module.prepare_data()

Max length of source sentence: 309
Max length of target sentence: 274


In [5]:
from bilingualmodule import BilingualModule

model = BilingualModule(
    tokenizer_src=data_module.tokenizer_src,
    tokenizer_tgt=data_module.tokenizer_tgt,
    seq_len=config.seq_len,
    d_model=config.d_model,
    lr=config.lr,
    weight_decay=config.weight_decay,
    eps=config.eps,
    label_smoothing=config.label_smoothing,
)

In [7]:
import sys
from pytorch_lightning.callbacks import (
    TQDMProgressBar,
    LearningRateMonitor,
    ModelCheckpoint,
)
from pytorch_lightning.loggers import TensorBoardLogger


class MyProgressBar(TQDMProgressBar):
    def init_validation_tqdm(self):
        bar = super().init_validation_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

    def init_predict_tqdm(self):
        bar = super().init_predict_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

    # def init_test_tqdm(self):
    #     bar = super().init_test_tqdm()
    #     if not sys.stdout.isatty():
    #         bar.disable = True
    #     return bar


logger = TensorBoardLogger("tb_logs", name="transformer_")

# training
trainer = pl.Trainer(
    log_every_n_steps=1,
    callbacks=[
        MyProgressBar(refresh_rate=1),
        LearningRateMonitor(logging_interval="epoch"),
        ModelCheckpoint(
            dirpath="ckpt_logs/transformer_",
            save_top_k=1,
            monitor="train_loss",
            mode="min",
            filename="model-{epoch:02d}-{train_loss:4f}",
            save_last=True,
        ),
    ],
    logger=logger,
    # precision=16,  # "16-mixed",
    accelerator="gpu",
    devices="auto",
    # strategy="ddp_notebook",
    check_val_every_n_epoch=1,
    # limit_train_batches=5,
    limit_val_batches=2,
    # limit_test_batches=1,
    #max_epochs=config.num_epochs,
    max_epochs=30,
)


# Uncomment the following line to train the model
#trainer.fit(model, datamodule=data_module)
trainer.fit(model, datamodule=data_module, ckpt_path="/content/drive/MyDrive/transformer/lightning_transformer/ckpt_logs/transformer_/last.ckpt")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/transformer/lightning_transformer/ckpt_logs/transformer_/last.ckpt


Max length of source sentence: 309
Max length of target sentence: 274


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type             | Params
----------------------------------------------
0 | bimodel  | Transformer      | 75.1 M
1 | criteria | CrossEntropyLoss | 0     
----------------------------------------------
75.1 M    Trainable params
0         Non-trainable params
75.1 M    Total params
300.532   Total estimated model params size (MB)
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint at /content/drive/MyDrive/transformer/lightning_transformer/ckpt_logs/transformer_/last.ckpt


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.
